In [40]:
import pandas as pd
import numpy as np
import ast
import altair as alt

In [41]:
rest = pd.read_csv("../buffet_restaurants.csv")
df = pd.read_csv("plot5_finalset.csv")
df.food_score = df.food_score.round(2)
df.service_score = df.service_score.round(2)
df = df.rename(columns={"star_binning":"AverageRating", "price":"Price"})
df["AverageRating"] = df["AverageRating"].astype("category")
df = df[["business_id", "AverageRating", "Price"]]

rest.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,gsicNDwrLH4WGeWMDcJv6w,Ahmed Indian Restaurant,"11301 S Orange Blossom Trl, Ste 104",Orlando,FL,32837,28.401633,-81.404314,3.5,146,1,"{'RestaurantsReservations': 'True', 'Restauran...","Indian, Pakistani, Bangladeshi, Restaurants, H...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
1,n4diWKcRvAFctwdRA6IPMg,Ahmed Indian Restaurant,10042 University Blvd,Orlando,FL,32817,28.596572,-81.243464,4.0,131,1,"{'ByAppointmentOnly': 'False', 'NoiseLevel': ""...","Restaurants, Halal, Indian, Pakistani, Banglad...","{'Monday': '11:30-21:0', 'Tuesday': '11:30-22:..."
2,riAMZdetrgWrwoUQUoJHRQ,Bachata Breeze Restaurant,5773 Curry Ford Rd,Orlando,FL,32822,28.523555,-81.308013,3.0,9,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Latin American, Restaurants, Caribbean, Puerto...","{'Monday': '11:0-17:0', 'Tuesday': '11:0-17:0'..."


In [42]:
df.head(3)

,business_id,AverageRating,Price
0,gsicNDwrLH4WGeWMDcJv6w,3,2
1,n4diWKcRvAFctwdRA6IPMg,3,2
2,ZCDaGq6eJfG5qB4n3b78Ag,3,2


In [43]:
attributes = ['RestaurantsDelivery', 'OutdoorSeating', 'WiFi', 'HappyHour',
                      'RestaurantsTakeOut', "RestaurantsReservations", "BusinessParking"]
attr_data = []
for row in range(len(rest)):
    single = []
    row_str = rest.loc[row, "attributes"]
    row_str = str(row_str)
    if not row_str or row_str == "nan":
        attr_data.append([0] * len(attributes))
        continue
    d = ast.literal_eval(row_str)
    for attr in attributes:
        if attr in d and (d[attr] == "True" or d[attr] == True or "free" in d[attr] or "True" in d[attr]):
            single.append(True)
        elif attr in d and (d[attr] == "False" or d[attr] == False):
            single.append(False)
        else:
            single.append(False)
    attr_data.append(single)

In [44]:
rest[attributes] = attr_data
rest = rest.merge(df, on="business_id")
rest = rest[["business_id", "name", "RestaurantsDelivery", "OutdoorSeating", "WiFi", "HappyHour",
             "RestaurantsTakeOut", "RestaurantsReservations", "BusinessParking", 
             "Price", "AverageRating"]]
rest

,business_id,name,RestaurantsDelivery,OutdoorSeating,WiFi,HappyHour,RestaurantsTakeOut,RestaurantsReservations,BusinessParking,Price,AverageRating
0,gsicNDwrLH4WGeWMDcJv6w,Ahmed Indian Restaurant,True,False,True,False,True,True,True,2,3
1,n4diWKcRvAFctwdRA6IPMg,Ahmed Indian Restaurant,True,False,True,True,True,True,False,2,3
2,ZCDaGq6eJfG5qB4n3b78Ag,Backlot Express,False,True,True,False,True,False,False,2,3
3,CHTiicer1vhhJ0AfjuEHxA,Biergarten Restaurant,False,True,True,False,False,True,False,3,3
4,CnNt_kUkrJ-kpgPPGbMG4A,Boi Brazil Churrascaria,False,False,False,False,False,True,True,2,3
...,...,...,...,...,...,...,...,...,...,...,...
68,3O8mfTK37HGnCgowSLG3bA,The Garden Gallery,False,True,True,False,True,False,False,2,3
69,6-nX4fFhEBZBOGPZGGl5ng,The Knife - Lake Buena Vista,True,False,True,False,True,True,True,3,2
70,Yttutdu1cYsHQU_nEu-ZDg,Tony's Brazilian Grill,False,False,True,True,True,True,False,2,3
71,5xjvDjN7s-FC00gkqAPqmA,Tropical Cabana,True,True,True,False,True,True,True,1,4


In [45]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [46]:
rest_df = pd.read_csv("../buffet_restaurants.csv")
reviews = pd.read_csv("../buffet_reviews.csv")
reviews = reviews.rename(columns={"stars":"review_stars"})
data = rest_df.merge(reviews, on="business_id")
aspect = pd.read_csv("../buffet_review_aspect.csv")
data = data.merge(aspect, on=["business_id","review_id"])
data["aspect"] = data["aspect"].str.lower()
data["description"] = data["description"].str.lower()
data.loc[data["description"]=="inferior", "polarity"] = -0.5
data = data[data["review_count"] > 6]
data["aspect"] = data["aspect"].astype(str)
data['aspect'] = data.aspect.apply(lambda x: lemmatizer.lemmatize(x))
lamb = data[data["aspect"] == "lamb"].drop_duplicates("business_id")[["business_id"]]
lamb["lamb"] = True
calamari = data[data["aspect"] == "calamari"].drop_duplicates("business_id")[["business_id"]]
calamari["calamari"] = True
crawfish = data[data["aspect"] == "crawfish"].drop_duplicates("business_id")[["business_id"]]
crawfish["crawfish"] = True
beer = data[data["aspect"] == "beer"].drop_duplicates("business_id")[["business_id"]]
beer["beer"] = True

chili = data[data["aspect"] == "chili"].drop_duplicates("business_id")[["business_id"]]
chili["chili"] = True
curry = data[data["aspect"] == "curry"].drop_duplicates("business_id")[["business_id"]]
curry["curry"] = True
sushi = data[data["aspect"] == "sushi"].drop_duplicates("business_id")[["business_id"]]
sushi["sushi"] = True
brownie = data[data["aspect"] == "brownie"].drop_duplicates("business_id")[["business_id"]]
brownie["brownie"] = True
pizza = data[data["aspect"] == "pizza"].drop_duplicates("business_id")[["business_id"]]
pizza["pizza"] = True

In [47]:
rest = rest.merge(lamb, on="business_id", how="left")
rest = rest.merge(calamari, on="business_id", how="left")
rest = rest.merge(crawfish, on="business_id", how="left")
rest = rest.merge(beer, on="business_id", how="left")
rest = rest.merge(chili, on="business_id", how="left")
rest = rest.merge(curry, on="business_id", how="left")
rest = rest.merge(sushi, on="business_id", how="left")
rest = rest.merge(brownie, on="business_id", how="left")
rest = rest.merge(pizza, on="business_id", how="left")
rest["lamb"] = rest["lamb"].fillna(False)
rest["calamari"] = rest["calamari"].fillna(False)
rest["crawfish"] = rest["crawfish"].fillna(False)
rest["beer"] = rest["beer"].fillna(False)
rest["chili"] = rest["chili"].fillna(False)
rest["curry"] = rest["curry"].fillna(False)
rest["sushi"] = rest["sushi"].fillna(False)
rest["brownie"] = rest["brownie"].fillna(False)
rest["pizza"] = rest["pizza"].fillna(False)

In [48]:
def price_func(price):
    if price == 1:
        new="$"
    elif price == 2:
        new="$$"
    elif price == 3:
        new="$$$"
    elif price == 4:
        new="$$$$"
    elif price == 5:
        new="$$$$$"
    else:
        new=0
    return new
rest['Price'] = df.apply(lambda x: price_func(x['Price']), axis=1)

In [49]:
rest.to_csv("../binary_characteristic_food.csv", index=False)

In [50]:
rest

,business_id,name,RestaurantsDelivery,OutdoorSeating,WiFi,HappyHour,RestaurantsTakeOut,RestaurantsReservations,BusinessParking,Price,AverageRating,lamb,calamari,crawfish,beer,chili,curry,sushi,brownie,pizza
0,gsicNDwrLH4WGeWMDcJv6w,Ahmed Indian Restaurant,True,False,True,False,True,True,True,$$,3,False,False,False,True,False,True,False,False,False
1,n4diWKcRvAFctwdRA6IPMg,Ahmed Indian Restaurant,True,False,True,True,True,True,False,$$,3,False,False,False,False,False,True,False,False,False
2,ZCDaGq6eJfG5qB4n3b78Ag,Backlot Express,False,True,True,False,True,False,False,$$,3,False,False,False,False,False,False,False,False,False
3,CHTiicer1vhhJ0AfjuEHxA,Biergarten Restaurant,False,True,True,False,False,True,False,$$$,3,False,False,False,True,False,False,False,False,False
4,CnNt_kUkrJ-kpgPPGbMG4A,Boi Brazil Churrascaria,False,False,False,False,False,True,True,$$,3,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,3O8mfTK37HGnCgowSLG3bA,The Garden Gallery,False,True,True,False,True,False,False,$$,3,False,False,False,False,False,False,False,True,False
69,6-nX4fFhEBZBOGPZGGl5ng,The Knife - Lake Buena Vista,True,False,True,False,True,True,True,$$$,2,False,False,False,True,False,False,False,False,True
70,Yttutdu1cYsHQU_nEu-ZDg,Tony's Brazilian Grill,False,False,True,True,True,True,False,$$,3,False,False,False,True,False,False,False,False,True
71,5xjvDjN7s-FC00gkqAPqmA,Tropical Cabana,True,True,True,False,True,True,True,$,4,False,False,False,True,False,False,False,False,False


In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(data=
#     go.Parcoords(
#         line = dict(color = rest['AverageRating'],
#                    colorscale = 'rainbow',
#                    showscale = True,
#                    cmin = 1,
#                    cmax = 4),
#         dimensions = list([
#             dict(tickvals = [1,2,3,4],
#                  ticktext = ['1','2','3','4'],
#                  label = 'AverageRating', values = rest['AverageRating']),
#             dict(tickvals = [0,1],
#                  ticktext = ['False','True'],
#                  visible = True,
#                  label = 'OutdoorSeating', values = rest['OutdoorSeating']),
#             dict(tickvals = [0,1],
#                  ticktext = ['False','True'],
#                  visible = True,
#                  label = 'WiFi', values = rest['WiFi']),
#             dict(tickvals = [0,1],
#                  ticktext = ['False','True'],
#                  visible = True,
#                  label = 'HappyHour', values = rest['HappyHour']),
#             dict(tickvals = [0,1],
#                  ticktext = ['False','True'],
#                  visible = True,
#                  label = 'RestaurantsReservations', values = rest['RestaurantsReservations']),
#             dict(tickvals = [0,1],
#                  ticktext = ['False','True'],
#                  visible = True,
#                  label = 'BusinessParking', values = rest['BusinessParking']),
#             dict(tickvals = [1,2,3,4],
#                  ticktext = ['$','$$','$$$','$$$$'],
#                  visible = True,
#                  label = 'Price', values = rest['Price'])])
#     )
# )
# fig.show()